# Imports

In [1]:
import pickle
from tqdm.notebook import tqdm

import re
import gc
import multiprocessing as mp
import os

# Load articles

In [2]:
def load_data(wiki_text_file: str):
    
    '''This function loads the text file that contains all the Spanish Wikipedia Articles'''
    
    lines = []
    for line in open(wiki_file):
        lines.append(line)
    data = ''.join(lines)
    
    return data

#####


wiki_file = '../DATA/eswiki-20200601-pages-articles-multistream.txt'
data_raw = load_data(wiki_file)

# Split articles

In [3]:
data_wiki_articles = data_raw.split('<page>')
print(f'Total # of Spanish Wikipedia articles: {len(data_wiki_articles)}')

Total # of Spanish Wikipedia articles: 3928922


# Filter articles

In [4]:
patterns = ['{{Ficha de artista musical', '|Discográfica', '|discográfica',
            '|voz', '|compañía discográfica', '|instrumento']
data_artists_articles = [pat for pat in data_wiki_articles if any(p in pat for p in patterns)]

patterns = ['{{Ficha de persona', '{{Ficha de artista musical']
data_artists_articles = [pat for pat in data_artists_articles if any(p in pat for p in patterns)]

patterns = ['{{Ficha de personaje']
data_artists_articles = [pat for pat in data_artists_articles if not any(p in pat for p in patterns)]


print(f'Total # of Spanish Wikipedia Musical Artists articles: {len(data_artists_articles)}')

del data_raw
del data_wiki_articles

gc.collect();

Total # of Spanish Wikipedia Musical Artists articles: 26807


# Extract article parts

In [5]:
def extract_header_and_body(data: list):

    '''This function extracts the article itself and the title of it'''
    
    data_artists = list()
    
    title_pattern = r'<title>[\s\S]*?</title>'
    id_pattern = r'<id>[\s\S]*?</id>'
    
    full_body_pattern = r'<text[\s\S]*?>[\s\S]*?</text>'
    body_pattern = r'<text[\s\S]*?>'
    
    
    with tqdm(total=len(data)) as pbar:
        
        for d in data:

            data_artist = dict()
            
            data_artist['_titulo_'] = re.search(title_pattern, d).group()[7:-8]
            data_artist['_id_'] = re.search(id_pattern, d).group()[4:-5]
            
            full_body = re.search(full_body_pattern, d).group()
            body_header = re.search(body_pattern, full_body).group()
            data_artist['body'] = full_body.split(body_header)[1]
            data_artist['body'] = data_artist['body'].split('</text>')[0]
            
            data_artists.append(data_artist)
            pbar.update(1)
        
    return data_artists

#####


data = extract_header_and_body(data_artists_articles)

# Manual cleaning

In [16]:
def clean_particular_cases(data):

    '''This function cleans certain articles that aren't properly structured'''
        
    with tqdm(total=len(data)) as pbar:
        for d in data:
            if d['_titulo_']=='Daniela Carpio':
                d['body'] = d['body'].replace('{{{{',
                                                   '{{')

            elif d['_titulo_']=='Ski Mask the Slump God':
                d['body'] = d['body'].replace('* escritor de canciones',
                                              '* escritor de canciones}}')

            elif d['_titulo_']=='Marcus &amp; Martinus':
                d['body'] = d['body'].replace('marcusandmartinus.com/',
                                              'marcusandmartinus.com/]]')

            elif d['_titulo_']=='Virgil Popa':
                d['body'] = d['body'].replace('Virgil Popa.jpg',
                                              'Virgil Popa.jpg}}')

            elif d['_titulo_']=='Rangel':
                d['body'] = d['body'].replace('= [[elcartelurbano]],',
                                              '= [[elcartelurbano]]}}')

            elif d['_titulo_']=='Jennifer Holliday':
                d['body'] = d['body'].replace('10|1960|edad',
                                              '10|1960|edad}}')

            elif d['_titulo_']=='Infinite (grupo musical)':
                d['body'] = d['body'].replace('[[Hoya (Cantante)|Hoya]]',
                                              '[[Hoya (Cantante)|Hoya]]}}')

            elif d['_titulo_']=='Coro de la Generalidad Valenciana':
                d['body'] = d['body'].replace('[[Francisco Hervás]]&lt;br /&gt;',
                                              '[[Francisco Hervás]]&lt;br /&gt;}}')

            elif d['_titulo_']=='Boddega':
                d['body'] = d['body'].replace('Roberto Jijón, Carlos del Campo',
                                              'Roberto Jijón, Carlos del Campo}}')

            elif d['_titulo_']=='Andrea Bocelli':
                d['body'] = d['body'].replace('|idioma=español{{',
                                              '|idioma=español}}')
                
            elif d['_titulo_']=='V8 (banda)':
                d['body'] = d['body'].replace('br[[Almafuerte]]',
                                              'br>[[Almafuerte]]')  
                
            elif d['_titulo_']=='Panzer (banda española)':
                d['body'] = d['body'].replace('[[Chapa',
                                              'Chapa')
                
            elif d['_titulo_']=='Jeon So-mi':
                d['body'] = d['body'].replace('[[Twice]',
                                              '[[Twice]]')    

            elif d['_titulo_']=='Ville Valo':
                d['body'] = d['body'].replace('|Género:',
                                              '|Género =')
                d['body'] = d['body'].replace('|Años activo:',
                                              '|Años activo =')
                
            elif d['_titulo_']=='Shakira':
                d['body'] = d['body'].replace('nacionalidad    [[colombiana]]',
                                              'nacionalidad =   [[colombiana]]')
            
            elif d['_titulo_']=='Mark Kozelek':
                d['body'] = d['body'].replace('nombre de nacimientol',
                                              'nombre de nacimiento')
                
            elif d['_titulo_']=='T-Bone (rapero)':
                d['body'] = d['body'].replace('nombre de nacimientol',
                                              'nombre de nacimiento')
            
            elif d['_titulo_']=='Juan y Junior':
                d['body'] = d['body'].replace('|Nacimiento: ',
                                              '|Nacimiento= ')
            
            elif d['_titulo_']=='Yurena':
                d['body'] = d['body'].replace('lugar de nacimiento [[Santurce]]',
                                              'lugar de nacimiento = [[Santurce]]')
            
            elif d['_titulo_']=='Tobias Sammet':
                d['body'] = d['body'].replace('años activo:',
                                              'años activo=')
            
            elif d['_titulo_']=='Jacoby Shaddix':
                d['body'] = d['body'].replace('|Años activo:',
                                              '|Años activo=')
            
            elif d['_titulo_']=='Henkka Seppälä':
                d['body'] = d['body'].replace('|ocupación [[',
                                              '|ocupación = [[')
                
            elif d['_titulo_']=='Amaya Saizar':
                d['body'] = d['body'].replace('|lugar de nacimiento País Vasco',
                                              '|lugar de nacimiento = País Vasco')
              
            elif d['_titulo_']=='The Age of Information':
                d['body'] = d['body'].replace('| Estilo           - ',
                                              '| Estilo           =')
                
            elif d['_titulo_']=='Janne Puurtinen':
                d['body'] = d['body'].replace('|Años activo:',
                                              '|Años activo=')
            
            elif d['_titulo_']=='Víctor García (cantante)':
                d['body'] = d['body'].replace('|lugar de nacimiento ',
                                              '|lugar de nacimiento=')
            
            elif d['_titulo_']=='Orianthi Panagaris':
                d['body'] = d['body'].replace('pareja\n',
                                              'pareja=')

            elif d['_titulo_']=='Michael Romeo':
                d['body'] = d['body'].replace('|Género:',
                                              '|Género=')
            
            elif d['_titulo_']=='Dj Paul Elstak':
                d['body'] = d['body'].replace('|género',
                                              '|género=')
            
            elif d['_titulo_']=='Roy Khan':
                d['body'] = d['body'].replace('|&lt;nowiki&gt;[&lt;/nowiki&gt;|Nombre_de_nacimiento',
                                              '|Nombre_de_nacimiento')
            
            elif d['_titulo_']=='Fenriz':
                d['body'] = d['body'].replace('|&lt;nowiki&gt;[&lt;/nowiki&gt;|Nombre_de_nacimiento',
                                              '|Nombre_de_nacimiento')
            
            elif d['_titulo_']=='Patryk Dominik Sztyber':
                d['body'] = d['body'].replace('|&lt;nowiki&gt;[&lt;/nowiki&gt;|Nombre_de_nacimiento',
                                              '|Nombre_de_nacimiento')
            
            elif d['_titulo_']=='Derek Roddy':
                d['body'] = d['body'].replace('|&lt;nowiki&gt;[&lt;/nowiki&gt;|Nombre_de_nacimiento',
                                              '|Nombre_de_nacimiento')
                
                
            pbar.update(1)
            
    return data

#####


data = clean_particular_cases(data)

In [15]:
for d in data:
    if d['_titulo_']== :
        print(d['body'])

{{Ficha de persona
|nombre = Derek Roddy
|imagen = 
|tamaño de imagen = 
|pie de imagen = 
|&lt;nowiki&gt;[&lt;/nowiki&gt;|Nombre_de_nacimiento = Derek Roddy
|alias = One Take
|fecha de nacimiento = Carolina de Sur, {{bandera|Estados Unidos}} {{edad|28|08|1972}}
|fecha de fallecimiento = 
|instrumento = [[batería (instrumento musical)|batería]]
|género = [[Death metal]], [[blues]], [[jazz]]
|ocupación = [[Baterista]]
|años activo = [[1977]] - presente
|compañía discográfica = [[Pavement Records|Pavement]], Nightsky, Olympic, [[Relapse Records|Relapse]], Martyr, [[Karmageddon Media|Hammerheart]], [[Earache Records|Earache]]
|relacionados = [[Aurora Borealis]]&lt;br /&gt;[[Council of the Fallen]]&lt;br /&gt;[[Divine Empire]]&lt;br /&gt;[[Hate Eternal]]&lt;br /&gt;[[Malevolent Creation]]&lt;br /&gt;[[Nile]]&lt;br /&gt;[[Serpent's Rise]]&lt;br /&gt;[[Today Is the Day]]&lt;br /&gt;[[Traumedy]]
|página web = [http://www.derekroddy.com/ www.derekroddy.com]
}}

'''Derek &quot;One Take&quot; Ro

In [17]:
def remove_particular_cases(data, to_remove_l):
    
    '''This function removes certain articles that aren't related to musical artists'''
    
    ind_to_remove_l = list()
    
    with tqdm(total=len(data)) as pbar:
        for ind, d in enumerate(data):
            if d['_titulo_'] in to_remove_l:
                ind_to_remove_l.append(ind)
                
            pbar.update(1)

    for ind in ind_to_remove_l[::-1]:
        data.pop(ind)
        
      
    return data

#####


titulos_to_remove = ['Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Protección de artículos/2018/04',
                     'Plantilla:Ficha de persona/doc',
                     'Wikipedia:Café/Portal/Archivo/Propuestas/2009/09',
                     'Wikipedia:Consultas de borrado/Giancarlo Monsalve',
                     'Gauvain Sers',
                     'Wernher von Braun',
                     'Raymond Kurzweil',
                     'Tomás Luis de Victoria',
                     'Cristóbal de Morales',
                     'Francisco Guerrero', 
                     'Juan Cabanilles',
                     'Robert Moog',
                     'Diego Ortiz',
                     'Alejandro García Villalón («Virulo»)',
                     'Jean-Pierre Christin',
                     'Juan Navarro Hispalensis',
                     'Keiji Fujiwara',
                     'Phil Zimmermann']

data = remove_particular_cases(data, titulos_to_remove)

In [18]:
print(f'Total # of Spanish Wikipedia Musical Artists articles: {len(data)}')

Total # of Spanish Wikipedia Musical Artists articles: 26789


# Save the articles

In [19]:
pickle.dump(data, open( "../Data/data_v1.p", "wb" ) )